In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import math
import seaborn as sns
import os

# %config IPCompleter.greedy=True
%load_ext autoreload
%autoreload 2

sc.settings.verbosity = 0
sc.logging.print_header()
sns.set_context("paper")

scanpy==1.6.0 anndata==0.7.4 umap==0.4.3 numpy==1.18.2 scipy==1.4.1 pandas==1.0.3 scikit-learn==0.22.2.post1 statsmodels==0.10.1 python-igraph==0.8.0 louvain==0.6.1 leidenalg==0.8.0


In [2]:
# import local module containing misc code, helps keep notebooks clean from commonly used functions
import sys
sys.path.insert(0, '/scratchfs/cherring/brain_maturation/analysis/submittal_final/')
import new_misc_code as nmc 

## **Pull all significant devDEGs from logTMMs data pickel**

In [3]:
# load log2 TMM pseudo-bulked data from limma-voom
# logTMMs.pkl was saved in 11__dev-DEGs_age-trend-fits_rate-of-change.ipynb
logTMMs = nmc.load_obj( "../data/logTMMs.pkl")

In [4]:
# index genes in logTMMs dataframes are devDEGs
sig_genes = {}
for traj_itr, df_itr in logTMMs.items():
    sig_genes[traj_itr] = df_itr.index.values

In [5]:
for iii, jjj in sig_genes.items():
    print( iii, jjj.shape)

Astro (3497,)
ID2 (3755,)
L2-3_CUX2 (8842,)
L4_RORB (8885,)
L5-6_THEMIS (6924,)
L5-6_TLE4 (7239,)
LAMP5_NOS1 (2514,)
Micro (58,)
Oligo (2540,)
OPC (1321,)
PV (5558,)
PV_SCUBE3 (2745,)
SST (4377,)
VIP (4624,)


## **Query GO terms for each set of devDEGs**

In [6]:
# read in adata, to use observational data for each batch
adata_fl_nm = "../data/post-gaba-wt-dev-traj.h5ad"
adata = sc.read( adata_fl_nm)

### Get background genes (genes with >5 counts) for GO term query

In [7]:
# make sure all genes have at least 5 counts, should have been already done
print( sum(adata.X.sum(0).A1>5)==adata.shape[1])
back_ground = adata.var_names.values.astype(str).tolist()

True


### Scanpy .var_names_make_unique() formats gene duplicate names in way that is not recognized by GProfiler 

In [8]:
# check how many gene names are changed by scanpy var_names_make_unique()
# any of the filtered_feature_bc_matrix.h5 from our study can be used here
# this can also be skipped without much change to the GO analysis
gene_file = "/dd_userdata/usrdat03/userdata/cherring/data/cellranger_outputs/brain_maturation_RNA/RL1777_2d_v3/outs/filtered_feature_bc_matrix.h5"
test_adata = sc.read_10x_h5( gene_file)
raw_names = test_adata.var_names.values
test_adata.var_names_make_unique()
unq_names = test_adata.var_names.values.tolist()
current_names = adata.var_names.values.astype(str).tolist()

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [9]:
test_adata.var_names_make_unique()
unq_names = test_adata.var_names.values.tolist()

In [10]:
# function to change scanpy corrected gene names back to generic name, this does not change the result very much
def get_preunique_names( current_names, raw_names=raw_names, unq_names=unq_names):
    raw_current_names = np.zeros_like( current_names)
    for itr, gene_itr in enumerate( current_names):
        if gene_itr not in raw_names:
            raw_current_names[itr] = raw_names[unq_names.index(gene_itr)]
        else:
            raw_current_names[itr] = gene_itr
    print( f"{len( current_names) - sum( nmc.member_test( raw_current_names, current_names))} gene name changes")
    return( raw_current_names)

In [11]:
back_ground_raw = get_preunique_names( current_names)

12 name changes


In [12]:
from gprofiler import GProfiler
def query_genes( genes, bk_grd, p_thresh=0.05):
    # Gprofiler does not like Y_RNA, times out
    if 'Y_RNA' in genes:
        genes.remove('Y_RNA')
    # base url sets version used in our study
    gp = GProfiler( return_dataframe=True, base_url="https://biit.cs.ut.ee/gprofiler_archive3/e102_eg49_p15/")
    query_df = gp.profile( organism='hsapiens', query=genes, user_threshold=p_thresh, background=bk_grd)
    ####################################################
    query_df = query_df[np.in1d( query_df['source'].values, ['GO:MF','GO:BP'])]
    ####################################################
    # if you'd like less general GOs
#     parent_mk = np.array( [len( ii)>=0 for ii in query_df['parents'].values])
    return( query_df) #.loc[parent_mk])

In [13]:
sig_genes.keys()

dict_keys(['Astro', 'ID2', 'L2-3_CUX2', 'L4_RORB', 'L5-6_THEMIS', 'L5-6_TLE4', 'LAMP5_NOS1', 'Micro', 'Oligo', 'OPC', 'PV', 'PV_SCUBE3', 'SST', 'VIP'])

In [14]:
all_q_df = {}
for k_itr in sig_genes.keys():
    gene_itr = sig_genes[k_itr]
    raw_gene_itr = get_preunique_names( gene_itr)
    q_df = query_genes( raw_gene_itr.tolist(), bk_grd=back_ground_raw.tolist(), p_thresh=0.05)
    all_q_df[k_itr] = q_df

1 name changes
3 name changes
7 name changes
6 name changes
7 name changes
4 name changes
2 name changes
0 name changes
0 name changes
0 name changes
2 name changes
2 name changes
4 name changes
4 name changes


In [16]:
nmc.save_obj( all_q_df, "../data/devDEG_Gene-Ontology-hits.pkl")